## Use a Slider to interactively display channels from 3D cube.
From http://nbarbey.github.io/2011/07/08/matplotlib-slider.html

In [73]:
def cube_show_slider(cube, axis=2, **kwargs):
    """
    Display a 3d ndarray with a slider to move along the third dimension.

    Extra keyword arguments are passed to imshow
    """
    import matplotlib.pyplot as plt
    from matplotlib.widgets import Slider, Button, RadioButtons

    # check dim
    if not cube.ndim == 3:
        raise ValueError("cube should be an ndarray with ndim == 3")

    # generate figure
    fig = plt.figure()
    ax = plt.subplot(111)
    fig.subplots_adjust(left=0.25, bottom=0.25)

    # select first image
    s = [slice(0, 1) if i == axis else slice(None) for i in range(3)]
    im = cube[s].squeeze()

    # display image
    l = ax.imshow(im, **kwargs)

    # define slider
    axcolor = 'lightgoldenrodyellow'
    ax = fig.add_axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)

    slider = Slider(ax, 'Axis %i index' % axis, 0, cube.shape[axis] - 1,
                    valinit=0, valfmt='%i')

    def update(val):
        ind = int(slider.val)
        s = [slice(ind, ind + 1) if i == axis else slice(None)
                 for i in range(3)]
        im = cube[s].squeeze()
        l.set_data(im, **kwargs)
        fig.canvas.draw()

    slider.on_changed(update)

    plt.show()

In [1]:
from astropy.io import fits
from spectral_cube import SpectralCube
cube = "../nro_maps/12CO_20161002_FOREST-BEARS_spheroidal_xyb_grid7.5_0.099kms.fits"
spec_cube = SpectralCube.read(cube)
hdu = fits.open(cube)[0]
spec_cube.spectral_axis[18]

/Users/jesse/carma/spectral-cube/spectral_cube/spectral_cube.py:238: UserWarning: radio_beam is not installed. No beam can be created.
  warnings.warn("radio_beam is not installed. No beam "
/Users/jesse/carma/spectral-cube/spectral_cube/spectral_cube.py:246: UserWarning: Could not parse beam information from header.  Exception was: UnboundLocalError("local variable 'Beam' referenced before assignment",)
  "  Exception was: {0}".format(ex.__repr__()))


<Quantity -1168.5 m / s>

## Use Aplpy with a Slider to make a nicer channel slicer.

In [4]:
import shells
shell_list = shells.get_shells()
n = 8
shell = shell_list[n]

In [26]:
from aplpy import FITSFigure
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider, Button, RadioButtons

cube = "../nro_maps/12CO_20161002_FOREST-BEARS_spheroidal_xyb_grid7.5_0.099kms.fits"
chan = 100

title = "Shell " + str(n+1) + " - 12CO"
pad_factor = 2.
stretch = 'linear'
ra = shell.ra.value
dec = shell.dec.value
radius = shell.radius.value

def channel_slicer(cube=None, ra=None, dec=None, radius=None,
                   title=None, pad_factor=2., chan_init=0,
                   stretch='linear', circle_color='blue',
                   circle_style='-'):
    spec_cube = SpectralCube.read(cube)
    vel_list = spec_cube.spectral_axis

    #For the auto color scaling to the min and max intensities in the zoomed region.
    ra_grid = spec_cube.spatial_coordinate_map[1].to(u.deg).value
    dec_grid = spec_cube.spatial_coordinate_map[0].to(u.deg).value
    #shell_mask = (ra_grid - ra) ** 2. + (dec_grid - dec) ** 2. < (radius) ** 2.
    subcube_mask = (abs(ra_grid - ra) < radius * pad_factor) &\
               (abs(dec_grid - dec) < radius * pad_factor)
    sub_cube = spec_cube.with_mask(subcube_mask).minimal_subcube()

    #auto color scaling
    subcube_pixels = sub_cube[chan].value
    vmin, vmax = np.nanmin(subcube_pixels), np.nanmax(subcube_pixels)
    print(vmin, vmax)


    #center plot on the shell
    fig = plt.figure()
    subplot = FITSFigure(sub_cube.hdu, figure=fig, slices=[chan], auto_refresh=True)
    subplot.set_title("{} @ {}".format(title, vel_list[chan]))
    #subplot.recenter(ra, dec, radius*pad_factor)

    #Make aplpy grayscale plot with nice color scaling/stretch
    subplot.show_grayscale(stretch=stretch, vmin=vmin, vmax=vmax)     

    subplot.tick_labels.set_yformat("dd:mm")
    subplot.tick_labels.set_xformat("hh:mm")
    subplot.tick_labels.set_style('plain')

#Set up slider to change spectral channels.
    axcolor = 'lightgoldenrodyellow'
    ax = fig.add_axes([0.25, 0.95, 0.65, 0.03], axisbg=axcolor)
    slider = Slider(ax, 'Channel', 0, hdu.data.shape[0] - 1,
                    valinit=chan, valfmt='%i')

    vel_label = subplot.add_label(0.9,0.9,"{}".format(vel_list[chan]), relative=True, layer='vel_label')

    #Show the shell as circle.
    subplot.show_circles(ra, dec, radius, edgecolor=circle_color)

    def update(val):
        ind = int(slider.val)
        
        #subplot = FITSFigure(hdu, figure=fig, slices=[chan], auto_refresh=True)
        #subplot.recenter(ra, dec, radius*pad_factor)
        #subplot.
        subplot.set_title("{} @ {}".format(title, vel_list[ind])) 
        subcube_pixels = sub_cube[ind].value
        vmin, vmax = np.nanmin(subcube_pixels), np.nanmax(subcube_pixels)
        subplot.image.set_clim(vmin, vmax)
        subplot.image.set_data(sub_cube[ind].hdu.data)
        
        #fig.canvas.draw()
        
        #subplot.show_grayscale(stretch=stretch, vmin=vmin, vmax=vmax)
        #subplot.set_data()
        #fig.canvas.draw()

#def write_velmin():
#    current_chan = int(slider.val)
#def write_velmax():
#    current_chan = inte(slider.val)
    
        
    slider.on_changed(update)

#button_velmin.on_click(write_velmin)
#button_velmax.on_click(write_velmax)

    plt.show()

In [27]:
channel_slicer(cube=cube, ra=ra, dec=dec, radius=radius, title=title)

/Users/jesse/carma/spectral-cube/spectral_cube/spectral_cube.py:238: UserWarning: radio_beam is not installed. No beam can be created.
  warnings.warn("radio_beam is not installed. No beam "
/Users/jesse/carma/spectral-cube/spectral_cube/spectral_cube.py:246: UserWarning: Could not parse beam information from header.  Exception was: UnboundLocalError("local variable 'Beam' referenced before assignment",)
  "  Exception was: {0}".format(ex.__repr__()))


12.7157 26.7037


//anaconda/lib/python3.4/site-packages/matplotlib/artist.py:221: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)
